In [46]:

import ase
import numpy as np
from ase.visualize import view
from ase.io import read, write
from ase.io import cif, vasp, lammpsdata
from ase.build import make_supercell, sort, add_adsorbate, add_vacuum
from ase.build import bulk
import nglview

In [47]:
#second_slab = supercell.copy()

In [48]:
#trans = np.zeros_like(supercell.positions)
#trans[:,2] += (20 + supercell.cell[2][2])

In [49]:
#second_slab.set_positions(second_slab.positions + trans)

In [50]:
#combined_slabs = supercell + second_slab

In [51]:
#combined_slabs.cell[2][2] += (20 +supercell.cell[2][2])

In [52]:
!pwd

/store/Terry_project_production/system_prep/Reproduce_original_NiAl_smaller


In [53]:
NiPrimitive = bulk('Ni', 'fcc', a=3.528, orthorhombic=True)
NiPrimitive.cell

Cell([2.4946727240261395, 2.4946727240261395, 3.528])

In [54]:
NiPrimitive_a_b = NiPrimitive.cell[0][0]
NiPrimitive_c = NiPrimitive.cell[2][2]

In [55]:
v = view(NiPrimitive, viewer='ngl')
v

In [56]:
target_height  = 101.8
repeats_ni_a_b =  round(target_height/NiPrimitive_a_b)
repeats_ni_a_b

41

In [57]:
target_width_ni = 56
repeats_ni_c = 2*round(target_width_ni/NiPrimitive_c)
repeats_ni_c

32

In [58]:
AlPrimitive = bulk('Al', 'fcc', a=4.064, orthorhombic=True)
AlPrimitive.cell

Cell([2.873681958742129, 2.873681958742129, 4.064])

In [59]:
AlPrimitive_a_b = AlPrimitive.cell[0][0]
AlPrimitive_c = AlPrimitive.cell[2][2]

In [60]:
repeats_al_a_b =  round(target_height/AlPrimitive_a_b)
repeats_al_a_b

35

In [61]:
target_width_al = 28
repeats_al_c = 2*round(target_width_al/AlPrimitive_c) 
repeats_al_c 

14

In [62]:
nisalb = ase.build.fcc100("Ni", (repeats_ni_a_b, repeats_ni_a_b, repeats_ni_c), a=3.528, vacuum=1.0, orthogonal=True, periodic=True)

In [63]:
nisalb.cell

Cell([102.28158168507174, 102.28158168507174, 56.684])

In [64]:
v = view(nisalb, viewer='ngl')
v

In [65]:
alsalb = ase.build.fcc100("Al", (repeats_al_a_b, repeats_al_a_b, repeats_al_c), a=4.064, vacuum=1.0, orthogonal=True, periodic=True)

In [66]:
alsalb.cell

Cell([100.57886855597452, 100.57886855597452, 28.416])

In [67]:
alsal_2 = alsalb.copy()

In [68]:
trans = np.zeros_like(alsal_2.positions)
trans[:,2] += (20 + nisalb.cell[2][2])
alsal_2.set_positions(alsal_2.positions + trans)
combined_slabs = alsal_2 + nisalb
combined_slabs.cell[2][2] += (20 +nisalb.cell[2][2])
combined_slabs.write("gapped.xyz")

/home/hmacdope/anaconda3/envs/terry/lib/python3.9/site-packages/ase/io/extxyz.py:302: UserWarning: Skipping unhashable information adsorbate_info
  warnings.warn('Skipping unhashable information '


In [69]:
v = view(combined_slabs, viewer='ngl')
v

In [70]:
wafer = ase.build.stack(nisalb, alsalb, maxstrain=None)

In [71]:
cap_left = alsalb

In [72]:
reverse_wafer = ase.build.stack(nisalb,alsalb, maxstrain=None)

In [73]:
cap_right = nisalb

In [74]:
v = view(wafer, viewer='ngl')
v

In [75]:
v = view(reverse_wafer, viewer='ngl')
v

In [76]:
combined_wafer_left = ase.build.stack(wafer,wafer, maxstrain=None)
combined_wafer_left = ase.build.stack(cap_left,combined_wafer_left, maxstrain=None)


In [77]:
v = view(combined_wafer_left, viewer='ngl')
v

In [79]:
combined_wafer_right = ase.build.stack(reverse_wafer,reverse_wafer, maxstrain=None)
combined_wafer_right = ase.build.stack(combined_wafer_right, cap_right, maxstrain=None)


In [80]:
v = view(combined_wafer_right, viewer='ngl')
v

In [81]:
combined_including_al2O3 = ase.build.stack(combined_wafer_left,combined_slabs, maxstrain=None)


In [82]:
v = view(combined_including_al2O3, viewer='ngl')
v

In [83]:
combined_including_al2O3_both_sides = ase.build.stack(combined_including_al2O3,combined_wafer_right, maxstrain=None)


In [84]:
v = view(combined_including_al2O3_both_sides, viewer='ngl')
v

In [85]:
combined_including_al2O3_both_sides.write("full_slab.xyz")

In [86]:
lammpsdata.write_lammps_data("full_slab.data", combined_including_al2O3_both_sides)
combined_including_al2O3_both_sides.write("full_slab.xyz")

In [87]:
formula = combined_including_al2O3_both_sides.get_chemical_formula()
formula

'Al102900Ni322752'

In [88]:
 import re
numbers = re.findall(r'\d+', formula)


In [89]:
Al = int(numbers[0])
Ni = int(numbers[1])

In [90]:
Al/Ni

0.3188206424747174

In [91]:
Al + Ni

425652